In [1]:
from __future__ import annotations

import os 

from torch.utils.data import random_split

from mmpfn.datasets.pad_ufes_20 import PADUFES20Dataset

import os 
import torch 
import numpy as np 
import pandas as pd

from sklearn.metrics import accuracy_score
from mmpfn.models.mmpfn_v2 import MMPFNClassifier
from mmpfn.models.mmpfn_v2.constants import ModelInterfaceConfig
from mmpfn.models.mmpfn_v2.preprocessing import PreprocessorConfig
from mmpfn.scripts_finetune_mm.finetune_mmpfn_main import fine_tune_mmpfn

/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
/home/wall/anaconda3/envs/mmpfn2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# data_path = os.path.join(os.getenv('HOME'), "workspace/works/tabular_image/MultiModalPFN/mmpfn/data/pad_ufes_20")
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/pad_ufes_20")
dataset = PADUFES20Dataset(data_path)
# _ = dataset.get_images()
_ = dataset.get_embeddings()

Load embeddings from embeddings/pad_ufes_20/pad_ufes_20_dinov3.pt


In [ ]:
accuracy_scores = []
for seed in range(1):
    torch.manual_seed(seed)
    train_len = int(len(dataset) * 0.8)
    test_len = len(dataset) - train_len

    train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

    X_train = train_dataset.dataset.x[train_dataset.indices]
    y_train = train_dataset.dataset.y[train_dataset.indices]
    X_test = test_dataset.dataset.x[test_dataset.indices]
    y_test = test_dataset.dataset.y[test_dataset.indices]
    image_train = train_dataset.dataset.embeddings[train_dataset.indices]
    image_test = test_dataset.dataset.embeddings[test_dataset.indices]
    
    for i in range(X_train.shape[1]):
        col = X_train[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1
    for i in range(X_test.shape[1]):
        col = X_test[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1

    torch.cuda.empty_cache()

    save_path_to_fine_tuned_model = "./checkpoints/finetuned_mmpfn_pad_ufes_20.ckpt"
    
    fine_tune_mmpfn(
        # path_to_base_model="auto",
        save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
        # Finetuning HPs
        time_limit=60,
        finetuning_config={"learning_rate": 0.00001, "batch_size": 1, "max_steps": 100},
        validation_metric="log_loss",
        # Input Data
        X_train=pd.DataFrame(X_train),
        image_train=image_train,
        y_train=pd.Series(y_train),
        categorical_features_index=None,
        device="cuda",  # use "cpu" if you don't have a GPU
        task_type="multiclass",
        # Optional
        show_training_curve=False,  # Shows a final report after finetuning.
        logger_level=0,  # Shows all logs, higher values shows less
        freeze_input=True,  # Freeze the input layers (encoder and y_encoder) during finetuning
        mixer_type='MGM+CQAM', # MGM MGM+CQAM
        mgm_heads=128,
        cqam_heads=12,
    )

    # disables preprocessing at inference time to match fine-tuning
    no_preprocessing_inference_config = ModelInterfaceConfig(
        FINGERPRINT_FEATURE=False,
        PREPROCESS_TRANSFORMS=[PreprocessorConfig(name='none')]
    )

    # Evaluate on Test Data
    model_finetuned = MMPFNClassifier(
        model_path=save_path_to_fine_tuned_model,
        inference_config=no_preprocessing_inference_config, 
        ignore_pretraining_limits=True,
        mixer_type='MGM+CQAM', # MGM MGM+CQAM
        mgm_heads=128,
        cqam_heads=12,
    )

    clf_finetuned = model_finetuned.fit(X_train, image_train, y_train)
    acc_score = accuracy_score(y_test, clf_finetuned.predict(X_test, image_test))
    print("accuracy_score (Finetuned):", acc_score)
    accuracy_scores.append(acc_score)

Fine-tuning Steps:   8%|▊         | 8/100 [00:18<03:53,  2.54s/it, Best Val. Loss=0.541, Best Val. Score=-0.541, Training Loss=0.618, Val. Loss=0.567, Patience=43, Utilization=0, Grad Norm=6.88][2025-10-07 22:49:33,876] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  20%|██        | 20/100 [00:45<03:06,  2.33s/it, Best Val. Loss=0.541, Best Val. Score=-0.541, Training Loss=0.498, Val. Loss=0.547, Patience=32, Utilization=0, Grad Norm=9]   [2025-10-07 22:50:00,871] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [04:03,  2.43s/it, Best Val. Loss=0.519, Best Val. Score=-0.519, Training Loss=0.403, Val. Loss=0.535, Patience=-48, Utilization=0, Grad Norm=10.5]                         
[2025-10-07 22:53:17,158] INFO - Initial Validation Loss: 0.5909833408115309 Best Validation Loss: 0.5194899919954977 Total Steps: 101 Best Step: 86 Total Time Spent: 247.18138122558594


accuracy_score (Finetuned): 0.8739130434782608


Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-10-07 22:53:26,579] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  28%|██▊       | 28/100 [01:17<03:37,  3.02s/it, Best Val. Loss=0.508, Best Val. Score=-0.508, Training Loss=0.425, Val. Loss=0.508, Patience=24, Utilization=0, Grad Norm=5.69][2025-10-07 22:54:44,197] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [04:36,  2.76s/it, Best Val. Loss=0.458, Best Val. Score=-0.458, Training Loss=0.444, Val. Loss=0.478, Patience=-48, Utilization=0, Grad Norm=13.4]                         
[2025-10-07 22:58:01,517] INFO - Initial Validation Loss: 0.6059744821076322 Best Validation Loss: 0.4583944388110993 Total Steps: 101 Best Step: 50 Total Time Spent: 278.7322461605072


accuracy_score (Finetuned): 0.8326086956521739


Fine-tuning Steps:   3%|▎         | 3/100 [00:04<03:41,  2.28s/it, Best Val. Loss=0.602, Best Val. Score=-0.602, Training Loss=0.435, Val. Loss=0.636, Patience=48, Utilization=0, Grad Norm=4.9][2025-10-07 22:58:15,138] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   9%|▉         | 9/100 [00:26<05:29,  3.62s/it, Best Val. Loss=0.578, Best Val. Score=-0.578, Training Loss=0.548, Val. Loss=0.589, Patience=43, Utilization=0, Grad Norm=9.01]

In [ ]:
# get mean and std of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)
print("Std Accuracy:", std_accuracy)

Mean Accuracy: 0.8560869565217392
Std Accuracy: 0.007826086956521745


In [ ]:
# 1/128/24
# Mean Accuracy: 0.8352173913043478
# Std Accuracy: 0.008630188365773572
# 1/128/16
# Mean Accuracy: 0.8400000000000001
# Std Accuracy: 0.02477116440564191
# 1/64/16
# Mean Accuracy: 0.837391304347826
# Std Accuracy: 0.01623317800225605
# 1/64/16
# Mean Accuracy: 0.8282608695652174
# Std Accuracy: 0.015736487887699012
# 1/64/8
# Mean Accuracy: 0.8273913043478262
# Std Accuracy: 0.016186530842596754
# 2/64/16
# Mean Accuracy: 0.8382608695652174
# Std Accuracy: 0.01893177483224106
# 2/128/16
# Mean Accuracy: 0.8382608695652174
# Std Accuracy: 0.01893177483224106
# Mean Accuracy: 0.8469565217391304
# Std Accuracy: 0.02403526080188738
# 2/128/8
# Mean Accuracy: 0.8482608695652173
# Std Accuracy: 0.015273189626304612
# 2/192/12
# Mean Accuracy: 0.8478260869565217
# Std Accuracy: 0.022717131265414168
# 2/128/12
# Mean Accuracy: 0.8486956521739131
# Std Accuracy: 0.017150506889839996
# 2/64/12
# Mean Accuracy: 0.8404347826086956
# Std Accuracy: 0.018981634741689644
# 1/128/12
# Mean Accuracy: 0.8417391304347825
# Std Accuracy: 0.015998582167803577
# MGM 12
# Mean Accuracy: 0.8330434782608694
# Std Accuracy: 0.01292701630201609
# MGM 8